### Data Processing

In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg
import random
import yfinance as yf
%matplotlib inline
from sklearn.svm import NuSVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
 
allowExternal = True
suppressMessages = False


In [44]:
alpha = .6
c = 10

### NuSVR Implmentation

In [45]:
columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
abalone_data = pd.read_csv('Data/abalone.data', header = None, names = columns)
abalone_data['Sex'] = pd.Categorical(abalone_data['Sex'], categories=['I','M','F'], ordered=True)

y = abalone_data['Rings']                   #Dependent
X = abalone_data.drop(columns=['Rings'])    #Independent



abalone_preprocessor = ColumnTransformer(
    transformers = [
        ("num", StandardScaler(),['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight']),
        ("cat", OneHotEncoder(drop='first'), ['Sex']),
    ]
)

abalone_pipeline = Pipeline([
    ("preprocessor",abalone_preprocessor),
    ("svr", NuSVR(C=c, nu=1-alpha,gamma="auto",kernel='linear', tol=1e-7))

])

abalone_pipeline.fit(X,y)
abalone_svr = abalone_pipeline.named_steps['svr']
#X_transformed = abalone_pipeline.named_steps['preprocessor'].transform(X)
#coefficients = np.dot(abalone_svr.dual_coef_,abalone_svr.support_vectors_).flatten()
coefficients = abalone_svr.coef_.flatten()
intercept = abalone_svr.intercept_

coefficients

array([ 0.01180033,  0.92385042,  0.62961541,  3.8785749 , -3.84235663,
       -1.08063668,  1.04415583, -0.63380008,  0.06901395])

### PSG Implentation

In [46]:
columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
abalone_data = pd.read_csv('Data/abalone.data', header = None, names = columns)
abalone_data['Sex'] = pd.Categorical(abalone_data['Sex'], categories=['I','M','F'], ordered=True)

y = abalone_data['Rings']                   #Dependent
X = abalone_data.drop(columns=['Rings'])    #Independent\

abalone_preprocessor = ColumnTransformer(
    transformers = [
        ("num", StandardScaler(),['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight']),
        ("cat", OneHotEncoder(drop='first'), ['Sex']),
    ]
)

X_transformed = abalone_preprocessor.fit_transform(X)

In [47]:
N = X_transformed.shape[0]
regularization_term = 1/(2*c*N)

x1 = np.ones((N,1))
matrix_scenarios_body = np.column_stack((x1,X_transformed, y.values.reshape(-1,1)))
header = ["x%d" % (i) for i in range(1,matrix_scenarios_body.shape[1])]
matrix_scenarios = [header + ['scenario_benchmark'], matrix_scenarios_body]

quadratic_size = X_transformed.shape[1]+1
matrix_quadratic_body = np.zeros((quadratic_size,quadratic_size))
matrix_quadratic_body[1:,1:] = regularization_term
matrix_quadratic = [["x%d" % (i+1) for i in range(quadratic_size)], matrix_quadratic_body]

In [48]:
problem_name_p = "problem_abalone"
problem_statement_p = "minimize\n\
%f" % (1-alpha) + "*cvar_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
+quadratic(matrix_quadratic)\n\
Value: \n\
var_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
Solver: van"
problem_dictionary_abalone = {'problem_name': problem_name_p, 
                             'problem_statement': problem_statement_p,
                             'matrix_scenarios': matrix_scenarios, 
                             'matrix_quadratic': matrix_quadratic}

# Optimization Output
res_primal = psg.psg_solver(problem_dictionary_abalone, allowExternal, suppressMessages)

Running solver
Reading problem formulation
Asking for data information
Getting data
      9.1% of scenarios is processed
100% of abs(matrix_scenarios) was read
100% of matrix_quadratic was read
Start optimization
Ext.iteration=0  Objective=0.514819248264E+01  Residual=0.000000000000E+00
Ext.iteration=105  Objective=0.116083971704E+01  Residual=0.000000000000E+00
Optimization is stopped
Solution is optimal
Calculating resulting outputs. Writing solution.
Objective: objective = 1.16083971704 [1.780351355229E-08]
Solver has normally finished. Solution was saved.
Problem: problem_1, solution_status = optimal
Timing: data_loading_time = 0.12, preprocessing_time = 0.01, solving_time = 0.03
Variables: optimal_point = point_problem_1
Objective: objective = 1.16083971704 [1.780351355229E-08]
Function: cvar_risk(0.6,abs(matrix_scenarios)) =  2.902069074337E+00
Function: quadratic(matrix_quadratic) =  1.208730318205E-05
Function: var_risk(0.6,abs(matrix_scenarios)) =  1.421649510634E+00
OK. Solve

#### PSG/CVaR Regression Coefficients

In [49]:
coeff = res_primal['point_problem_1'][1][1:]
coeff


array([ 0.01001566,  0.9235053 ,  0.62587867,  3.9435627 , -3.87465006,
       -1.10397277,  1.04142231, -0.63172853,  0.07084149])

#### NuSVR Coefficients

In [50]:
coefficients

array([ 0.01180033,  0.92385042,  0.62961541,  3.8785749 , -3.84235663,
       -1.08063668,  1.04415583, -0.63380008,  0.06901395])

#### NnSVR Intercept

In [51]:
intercept

array([9.96343375])

#### PSG/CVaR Regression Intercept

In [52]:
res_primal['point_problem_1'][1][0]

9.965103523524213

### Data Splitting

In [57]:
columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
abalone_data = pd.read_csv('Data/abalone.data', header = None, names = columns)
abalone_data['Sex'] = pd.Categorical(abalone_data['Sex'], categories=['I','M','F'], ordered=True)

y = abalone_data['Rings']                   #Dependent
X = abalone_data.drop(columns=['Rings'])    #Independent



abalone_preprocessor = ColumnTransformer(
    transformers = [
        ("num", StandardScaler(),['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight']),
        ("cat", OneHotEncoder(drop='first'), ['Sex']),
    ]
)

abalone_pipeline = Pipeline([
    ("preprocessor",abalone_preprocessor),
    ("svr", NuSVR(C=c, nu=1-alpha,gamma="auto",kernel='linear', tol=1e-7))

])

svr_coeff_vec = []
svr_interp_vec = []
cvar_coeff_vec = []
cvar_interp_vec = []




for _ in range(1000):
    # Split the data into testing and training, then average over the coefficents
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5)

    abalone_pipeline.fit(X_train,y_train)
    abalone_svr = abalone_pipeline.named_steps['svr']
    #X_transformed = abalone_pipeline.named_steps['preprocessor'].transform(X)
    #coefficients = np.dot(abalone_svr.dual_coef_,abalone_svr.support_vectors_).flatten()
    coefficients = abalone_svr.coef_.flatten()
    intercept = abalone_svr.intercept_

    svr_coeff_vec.append(coefficients)
    svr_interp_vec.append(intercept)


    # PSG
    X_transformed = abalone_preprocessor.fit_transform(X_train)

    N = X_transformed.shape[0]
    regularization_term = 1/(2*c*N)

    x1 = np.ones((N,1))
    matrix_scenarios_body = np.column_stack((x1,X_transformed, y_train.values.reshape(-1,1)))
    header = ["x%d" % (i) for i in range(1,matrix_scenarios_body.shape[1])]
    matrix_scenarios = [header + ['scenario_benchmark'], matrix_scenarios_body]

    quadratic_size = X_transformed.shape[1]+1
    matrix_quadratic_body = np.zeros((quadratic_size,quadratic_size))
    matrix_quadratic_body[1:,1:] = regularization_term
    matrix_quadratic = [["x%d" % (i+1) for i in range(quadratic_size)], matrix_quadratic_body]

    problem_name_p = "problem_abalone"
    problem_statement_p = "minimize\n\
    %f" % (1-alpha) + "*cvar_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
    +quadratic(matrix_quadratic)\n\
    Value: \n\
    var_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
    Solver: van"
    problem_dictionary_abalone = {'problem_name': problem_name_p, 
                                'problem_statement': problem_statement_p,
                                'matrix_scenarios': matrix_scenarios, 
                                'matrix_quadratic': matrix_quadratic}

    # Optimization Output
    res_primal = psg.psg_solver(problem_dictionary_abalone, allowExternal, suppressMessages)

    cvar_coeff_vec.append(res_primal['point_problem_1'][1][1:])
    cvar_interp_vec.append(res_primal['point_problem_1'][1][0])

Running solver
Reading problem formulation
Asking for data information
Getting data
      9.1% of scenarios is processed
100% of abs(matrix_scenarios) was read
100% of matrix_quadratic was read
Start optimization
Ext.iteration=0  Objective=0.521072796935E+01  Residual=0.000000000000E+00
Ext.iteration=95  Objective=0.119425166465E+01  Residual=0.000000000000E+00
Optimization is stopped
Solution is optimal
Calculating resulting outputs. Writing solution.
Objective: objective = 1.19425166465 [5.988649132149E-09]
Solver has normally finished. Solution was saved.
Problem: problem_1, solution_status = optimal
Timing: data_loading_time = 0.10, preprocessing_time = 0.01, solving_time = 0.02
Variables: optimal_point = point_problem_1
Objective: objective = 1.19425166465 [5.988649132149E-09]
Function: cvar_risk(0.6,abs(matrix_scenarios)) =  2.985577391291E+00
Function: quadratic(matrix_quadratic) =  2.070813543586E-05
Function: var_risk(0.6,abs(matrix_scenarios)) =  1.491844845726E+00
OK. Solver

In [54]:
cvar_ave_coeff = [sum(col)/len(col) for col in zip(*cvar_coeff_vec)]
svr_ave_coeff = [sum(col)/len(col) for col in zip(*svr_coeff_vec)]


In [55]:
svr_ave_coeff

[0.06663225645783867,
 0.8687068186372268,
 0.5926547226134126,
 3.735787346579565,
 -3.7932459034503108,
 -1.030707604870167,
 1.105807893445083,
 -0.6508626363564335,
 0.06483302365944914]

In [56]:
cvar_ave_coeff

[0.06725650581450035,
 0.8714386720709176,
 0.5905901438444047,
 3.8502966832228145,
 -3.849507755031685,
 -1.0591340455234934,
 1.0717762642893147,
 -0.6495582806691174,
 0.06489035500052491]

In [59]:
cvar_ave_interp = np.mean(cvar_interp_vec)
svr_ave_interp = np.mean(svr_interp_vec)

In [60]:
cvar_ave_interp

9.965986342499919

In [61]:
svr_ave_interp

9.96402642088119